# External Lab 

Here each question is of 1 mark.

# Multiple Linear Regression

## Problem Statement

Use Multiple Linear Regression to **predict the consumption of petrol** given relevant variables are the petrol tax, the per capita, income, the number of miles of paved highway, and the proportion of the population with driver's licenses.

## Dataset

There are 48 rows of data.  The data include:

      I,  the index;
      A1, the petrol tax;
      A2, the per capita income;
      A3, the number of miles of paved highway;
      A4, the proportion of drivers;
      B,  the consumption of petrol.

### Reference 

    Helmut Spaeth,
    Mathematical Algorithms for Linear Regression,
    Academic Press, 1991,
    ISBN 0-12-656460-4.

    S Weisberg,
    Applied Linear Regression,
    New York, 1980, pages 32-33.

## Question 1 - Exploratory Data Analysis

*Read the dataset given in file named **'petrol.csv'**. Check the statistical details of the dataset.*

**Hint:** You can use **df.describe()**

In [1]:
import numpy as np
import pandas as pd 
from scipy import stats

In [2]:
df = pd.read_csv('petrol.csv')
df.head()
df.describe()

,tax,income,highway,dl,consumption
count,48.000000,48.000000,48.000000,48.000000,48.000000
mean,7.668333,4241.833333,5565.416667,0.570333,576.770833
std,0.950770,573.623768,3491.507166,0.055470,111.885816
min,5.000000,3063.000000,431.000000,0.451000,344.000000
25%,7.000000,3739.000000,3110.250000,0.529750,509.500000
50%,7.500000,4298.000000,4735.500000,0.564500,568.500000
75%,8.125000,4578.750000,7156.000000,0.595250,632.750000
max,10.000000,5342.000000,17782.000000,0.724000,968.000000


# Question 2 - Cap outliers 

Find the outliers and cap them. (Use (Q1 - 1.5 * IQR) as the minimum cap and (Q3 + 1.5 * IQR) as the max cap. The decision criteria is you should consider the datapoints which only falls within this range. The data points which fall outside this range are outliers and the entire row needs to be removed

In [34]:
df1 = df
desc = df1.describe()
Q1 = desc.loc['25%']
Q3 = desc.loc['75%']
IQR = Q3 - Q1
mincap = Q1-1.5*IQR
maxcap = Q1+1.5*IQR
for i in range(df1.shape[1]):
    df1 =df1[(df1.iloc[:,i]> mincap[i]) & (df1.iloc[:,i] < maxcap[i])]
df1

,tax,income,highway,dl,consumption
3,7.5,4870,2351,0.529,414
4,8.0,4399,431,0.544,410
8,8.0,4447,8577,0.529,464
9,7.0,4512,8507,0.552,498
10,8.0,4391,5939,0.530,580
12,7.0,4817,6930,0.574,525
13,7.0,4207,6580,0.545,508
14,7.0,4332,8159,0.608,566
16,7.0,4206,8508,0.572,603
21,8.0,4983,602,0.602,540


In [35]:
df1.shape

(23, 5)

# Question 3 - Independent variables and collinearity 
Which attributes seems to have stronger association with the dependent variable consumption?

In [36]:
corr = df.corr()
columns =corr.columns.str.strip()
corr.columns = columns
corr[abs(corr.consumption) == max(abs(corr.consumption.head(4)))].index[0]

'dl'

### Observing the above correlation values between all the variables, we can see that there is stronger association between the number of drivers and consumption. And comparatively tax has an association in a negative way. 
Insights :
As tax increases the consumption decreases.
As number of drivers is more consumption is more

In [37]:
df.corr()

,tax,income,highway,dl,consumption
tax,1.000000,0.012665,-0.522130,-0.288037,-0.451280
income,0.012665,1.000000,0.050163,0.157070,-0.244862
highway,-0.522130,0.050163,1.000000,-0.064129,0.019042
dl,-0.288037,0.157070,-0.064129,1.000000,0.698965
consumption,-0.451280,-0.244862,0.019042,0.698965,1.000000


# Question 4 - Transform the dataset 
Divide the data into feature(X) and target(Y) sets.

In [38]:
df.columns = df.columns.str.strip()
df.columns
X = df.loc[:,['tax','dl']]
Y = df.loc[:,['consumption']]

# Question 5 - Split data into train, test sets 
Divide the data into training and test sets with 80-20 split using scikit-learn. Print the shapes of training and test feature sets.

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2, random_state = 1)
print(X.shape)
print(Y.shape)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(48, 2)
(48, 1)
(38, 2)
(38, 1)
(10, 2)
(10, 1)


# Question 6 - Build Model 
Estimate the coefficients for each input feature. Construct and display a dataframe with coefficients and X.columns as columns

In [40]:
from sklearn.linear_model import LinearRegression as lm
lm_model = lm()
lm_model.fit(X_train,y_train)
coef = pd.DataFrame(lm_model.coef_[0])
coef = coef.T
coef.columns =  X.columns
coef.rename(index={0:'coefficients'}, inplace=True)
coef

,tax,dl
coefficients,-35.846692,1250.743596


In [41]:
coeff_df = pd.DataFrame(lm_model.coef_[0], X.columns, columns=['Coefficient'])  
coeff_df  

,Coefficient
tax,-35.846692
dl,1250.743596


# R-Square 

# Question 7 - Evaluate the model 
Calculate the accuracy score for the above model.

####  1st method to Calculating R2 score 

In [42]:
lm_model.score(X_test, y_test)

0.5904039469252051

#### 2nd method to calculate R2 score 

In [55]:
y_pred = lm_model.predict(X_test)

In [56]:
ssr = np.sum((y_pred - y_test)**2)
sst = np.sum((y_test - np.mean(y_test))**2)
r2_score = 1 - (ssr/sst)
r2_score

consumption    0.590404
dtype: float64

# Question 8: Repeat the same Multi linear regression modelling by adding both Income and Highway features
Find R2 


In [30]:
df.columns = df.columns.str.strip()
df.columns
X = df.iloc[:,[0,1,2,3]]
Y = df.loc[:,['consumption']]

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2, random_state = 1)
print(X.shape)
print(Y.shape)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(48, 4)
(48, 1)
(38, 4)
(38, 1)
(10, 4)
(10, 1)


In [32]:
from sklearn.linear_model import LinearRegression as lm
lm_model1 = lm()
lm_model1.fit(X_train,y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [33]:
lm_model1.score(X_test, y_test)

0.6611179509319676

# Question 9: Print the coefficients of the multilinear regression model

In [22]:
coeff_df1 = pd.DataFrame(lm_model1.coef_[0], X.columns, columns=['Coefficient'])  
coeff_df1  

,Coefficient
tax,-32.003954
income,-0.075188
highway,-0.002137
dl,1355.827606


# Question 10 
In one or two sentences give reasoning on R-Square on the basis of above findings
Answer

In [ ]:
# So in our first model, 59.04% of the variability in Y can be explained using X and when we include Highway and income columns 
# in regression model, 66.11 % of the variability in Y can be explained using X. 